### 6. 다양한 데이터타입 다루기

- 불리언 타입
- 수치 타입
- 문자열 타입
- date와 timestamp 다루기
- null값 다루기
- 복합 데이터 타입
- 사용자 정의 함수

In [2]:
# 스파크 api는 공식문서를 참고할 것! 
# 함수는 많지만 SQL과 비슷한 구문이 많다.

df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('file:///home/ubuntu/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv')

In [3]:
print(df.printSchema())

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

None


In [4]:
# 프로그래밍 언어의 고유 데이터타입을 스파크 데이터 타입으로 변환해야함. 
# lit 함수 사용
from pyspark.sql.functions import lit
df.select(lit(5), lit('five'), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [5]:
# 불리언 데이터 타입 다루기
# and, or, true, false

from pyspark.sql.functions import col

df.where(col('InvoiceNo')!=536365).select('InvoiceNo', 'Description').show(5,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [6]:
# 불리언 필터는 차례대로 적용해야함
# 스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환 시킴. 
# 그 다음에 모든 필터를 처리
# or 구문을 사용하면 동일한 구문에 조건을 정의해야함
from pyspark.sql.functions import instr

priceFilter = col('UnitPrice')> 600
descripFilter = instr(df.Description, 'POSTAGE') >=1
df.where(df.StockCode.isin('DOT')).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [7]:
# 불리언 컬럼 기반 필터링
from pyspark.sql.functions import instr

DOTCodeFilter = col('StockCode') == 'DOT'
priceFilter = col('UnitPrice')> 600
descripFilter = instr(df.Description, 'POSTAGE') >=1
df.withColumn('isExpensive', DOTCodeFilter & (priceFilter | descripFilter)).where('isExpensive').select('unitPrice', 'isExpensive').show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [8]:
from pyspark.sql.functions import expr

df.withColumn('isExpensive', expr('NOT UnitPrice <= 250')).filter('isExpensive').select('Description', 'UnitPrice').show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [9]:
# 수치형 데이터
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col('Quantity') * col('UnitPrice'), 2) + 5
df.select(expr('CustomerId'), fabricatedQuantity.alias('realQuantity')).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [10]:
df.selectExpr('CustomerId', '(POWER((Quantity * UnitPrice), 2.0)+5) as realQuantity').show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [11]:
# round는 반올림
# bround는 반내림
from pyspark.sql.functions import lit, round, bround

df.select(round(lit('2.5')), bround(lit('2.5'))).show(5)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 5 rows



In [15]:
# 피어슨 상관계수 
from pyspark.sql.functions import corr

print(df.stat.corr('Quantity', 'UnitPrice'))
df.select(corr('Quantity', 'UnitPrice')).show(2)

-0.04112314436835551
+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [16]:
# 요약통계는 describe, 통계 스키마는 변경될 수 있기 때문에 콘솔 확인용으로만 사용.
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

In [18]:
# 정확한 수치를 확인하기 위해서는 직접 입력하는 방식을 쓰자
from pyspark.sql.functions import count, mean, stddev_pop, min, max

olName = 'UnitPrice'
quantileProbs = [0.5]
relError = 0.05

# 데이터의 백분위수를  계싼하거나 근사치 계산
df.stat.approxQuantile('UnitPrice', quantileProbs, relError)

[2.51]

In [21]:
# 교차표나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드
# 연산값이 너무 크면 보이지 않는다.
df.stat.crosstab('StockCode', 'Quantity').show()
df.stat.freqItems(['StockCode','Quantity']).show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [23]:
# 모든 로우에 고유 ID를 추가해준다.
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



In [24]:
# 대소문자 변환
# initcap-> 문자열에서 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변환
from pyspark.sql.functions import initcap
df.select(initcap(col('Description'))).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



In [27]:
# lower와 upper
from pyspark.sql.functions import lower, upper
df.select(col('Description'), lower(col('Description')), upper(lower(col('Description')))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



In [31]:
# 공백을 채워주는 함수
# lpad, ltrim, rpad, rtrim, trim
# lpad와 rpad는 문자열의 길이보다 작은 숫자를 넘기면 문자열의 오른쪽부터 제거됨. 

from pyspark.sql.functions import lit, ltrim, lpad, rtrim, rpad, lpad, trim

df.select(
          ltrim(lit('  hello  ')).alias('ltrim'),
          rtrim(lit('  hello  ')).alias('rtrim'),
          trim(lit('  hello  ')).alias('trim'),
          lpad(lit('hello'), 3, ' ').alias('lp'),
          rpad(lit('hello'), 3, ' ').alias('rp')).show(2)

+-------+-------+-----+---+---+
|  ltrim|  rtrim| trim| lp| rp|
+-------+-------+-----+---+---+
|hello  |  hello|hello|hel|hel|
|hello  |  hello|hello|hel|hel|
+-------+-------+-----+---+---+
only showing top 2 rows



In [32]:
# 정규표현식
from pyspark.sql.functions import regexp_replace

regex_string = 'BLACK|WHITE|RED|GREEN|BLUE'
df.select(regexp_replace(col('Description'), regex_string, 'COLOR').alias('color_ckean'),
         col('Description')).show(2)

+--------------------+--------------------+
|         color_ckean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [33]:
from pyspark.sql.functions import translate

df.select(translate(col('Description'), 'LEET', '1337'), col('Description')).show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [35]:
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(regexp_extract(col('Description'), extract_str, 1).alias('color_clean'), col('Description')).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [43]:
# instr로 값의 존재 여부 확인
from pyspark.sql.functions import instr
containsBlack = instr(col('Description'), 'BLACK') >= 1
containsWhite = instr(col('Description'), 'White') >= 1
df.withColumn('hasSimpleColor', containsBlack | containsWhite).where('hasSimpleColor').select('Description').show(5,False)

+---------------------------------+
|Description                      |
+---------------------------------+
|JUMBO  BAG BAROQUE BLACK WHITE   |
|EDWARDIAN PARASOL BLACK          |
|WOOD BLACK BOARD ANT WHITE FINISH|
|JUMBO  BAG BAROQUE BLACK WHITE   |
|BLACK/BLUE POLKADOT UMBRELLA     |
+---------------------------------+
only showing top 5 rows



In [44]:
# 동적으로 컬럼 생성하는 법
# locate 함수 활용
from pyspark.sql.functions import expr, locate

simpleColors = ['black', 'white','red','green', 'blue']
def color_locator(column, color_string):
    return locate(color_string.upper(), column).cast('boolean').alias('is_'+color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr('*'))

df.select(*selectedColumns).where(expr('is_white OR is_red')).select('Description').show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [46]:
# 날짜 데이터 다루기
# 두 가지 종류의 시간 관련 정보만 집중적으로 관리함 -> date, timestamp
# inferSchema가 있을 경우 최대한 정확하게 식별하려고 시도함. 
# 특이한 날짜 포맷이 있을 경우 포맷을 파악하고 트랜스포메이션 해줘야함. 

# 오늘 날짜 구하기
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10).withColumn('today', current_date()).withColumn('now', current_timestamp())
dateDF.createOrReplaceTempView('dataTable')
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [47]:
# 날짜 계산 함수
from pyspark.sql.functions import date_add, date_sub

dateDF.select(date_sub(col('today'), 5), date_add(col('today'),5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-09-21|        2020-10-01|
+------------------+------------------+
only showing top 1 row



In [53]:
# 날짜 차이 계산 함수

from pyspark.sql.functions import datediff, months_between, to_date

dateDF.withColumn('week_ago', date_sub(col('today'), 7))\
.select(datediff(col('week_ago'), col('today'))).show(1)

dateDF.select(to_date(lit('2016-01-01')).alias('start'), to_date(lit('2017-05-22')).alias('end'))\
.select(months_between(col('start'), col('end'))).show(1)

# to_date로 문자열을 날짜로 변환할 수 있음. 필요에 따라 날짜 포맷도 함께 지정할 수 있음

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [55]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn('date', lit('2017-01-01')).select(to_date(col('date'))).show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [56]:
# 날짜를 파싱할 수 없으면 에러 대신 null값을 반환함.
dateDF.select(to_date(lit('2016-20-12')), to_date(lit('2017-12-11'))).show(1)

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



In [58]:
from pyspark.sql.functions import to_date

dateFormat = 'yyyy-dd-MM'
cleanDateDF = spark.range(1).select(to_date(lit('2017-12-11'), dateFormat).alias('date'),
                                   to_date(lit('2017-20-12'), dateFormat).alias('date2'))
cleanDateDF.createOrReplaceTempView('dateTable2')
cleanDateDF.show(5)

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [59]:
# 날짜 포맷을 항상 지정해줘야함 
from pyspark.sql.functions import to_timestamp

cleanDateDF.select(to_timestamp(col('date'), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [61]:
# 날짜 비교
cleanDateDF.filter(col('date2')> lit('2017-12-12')).show()
cleanDateDF.filter(col('date2')> '2017-12-12').show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [64]:
# null 값 다루기
# 스파크에서는 빈 문자열이나 대체 값 대신 null을 사용해야 최적화를 수행할 수 있음 
# null값이 없어야 하는 컬럼에 null값이 존재하면 부정확한 결과를 초래하거나 디버깅하기 어려운 특이한 오류를 접할 수 있음.

from pyspark.sql.functions import coalesce

# null이 아닌 첫번째 값을 반환해주는 함수
df.select(coalesce(col('Description'), col('CustomerId'))).show(5, False)

+-----------------------------------+
|coalesce(Description, CustomerId)  |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



In [69]:
# ifnull: 첫번째 값이 null이면 두번째 반환, 반대는 첫번째 반환
# nullif: 두 값이 같으면 null 반환, 다르면 첫번째 값 반환
# nvl: 첫번째 값이 null이면 두번째 값 반환, 첫번째가 null이 아니면 첫번째 반환
# nvl2: 첫번째 값이 null이 아니면 두번째 값 반환, 그리고 첫번째 값이 null이면 세번째 인수로 지정된 값 반환

# drop으로 null을 가진 로우 제거
# any를 지정하면 로우의 컬럼값중 하나라도 null을 가지면 제거
# all은 모든 컬럼값이 null일 때 해당 로우 제거
df.na.drop()
df.na.drop('any')
df.na.drop('all')

# 배열 형태로 인수 전달도 가능
df.na.drop('all', subset=['StockCode', 'InvoiceNo'])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [74]:
# fill로 na 채우기
df.na.fill('All Null values become this string')
df.na.fill('all', subset=['StockCode', 'InvoiceNo'])
# map 방식
fill_cols_vals = {'StockCode':5, 'Description':'No_value'}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [75]:
# replace로 대체 가능
df.na.replace([""], ['UNKNOWN'],'Description')

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [77]:
# 복합데이터 다루기
# 구조체, 배열, 맵으로 이루어져 있음

# 쿼리문에서 다수의 컬럼을 괄호로 묶어 구조체를 만들 수 있음
from pyspark.sql.functions import struct

complexDF = df.select(struct('Description', 'InvoiceNo').alias('complex'))
complexDF.createOrReplaceTempView('complexDF')
complexDF.show(5)

+--------------------+
|             complex|
+--------------------+
|[WHITE HANGING HE...|
|[WHITE METAL LANT...|
|[CREAM CUPID HEAR...|
|[KNITTED UNION FL...|
|[RED WOOLLY HOTTI...|
+--------------------+
only showing top 5 rows



In [78]:
complexDF.select('complex.Description')
complexDF.select(col('complex').getField('Description'))

DataFrame[complex.Description: string]

In [79]:
complexDF.select('complex.*')

DataFrame[Description: string, InvoiceNo: string]

In [81]:
# 배열로 변환시키기
from pyspark.sql.functions import split
df.select(split(col('Description'), " ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [82]:
# 배열의 특정 값 알기
df.select(split(col('Description'), ' ').alias('array_col')).selectExpr('array_col[0]').show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [84]:
# 배열의 길이 파악
from pyspark.sql.functions import size
df.select(size(split(col('Description'), " "))).show(2)

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|                          5|
|                          3|
+---------------------------+
only showing top 2 rows



In [85]:
# 배열에 특정 값 유무 파악
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col('Description'), " "), 'WHITE')).show(2)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|                                        true|
|                                        true|
+--------------------------------------------+
only showing top 2 rows



In [87]:
# 복합 데이터 타입의 배열에 존재하는 모든 값을 로우로 변환하려면 explode 활용
from pyspark.sql.functions import split, explode

df.withColumn('splited', split(col('Description'), " "))\
.withColumn('exploded', explode(col('splited')))\
.select('Description', 'InvoiceNo', 'exploded').show(2)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
+--------------------+---------+--------+
only showing top 2 rows



In [90]:
# 컬럼의 키-값을 이용해서 맵으로 생성함. 배열과 동일한 방식으로 값을 선택할 수 있음
from pyspark.sql.functions import create_map

df.select(create_map(col('Description'), col('InvoiceNo')).alias('complex_map')).show(2, False)

+----------------------------------------------+
|complex_map                                   |
+----------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365]|
|[WHITE METAL LANTERN -> 536365]               |
+----------------------------------------------+
only showing top 2 rows



In [91]:
# 적합한 키를 이용해 조회 가능, 없으면 Null 반환
df.select(create_map(col('Description'), col('InvoiceNo')).alias('complex_map'))\
.selectExpr('complex_map["WHITE METAL LANTERN"]').show(2)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows



In [92]:
# map 타입은 분해해서 컬럼으로 변환할 수 있음
df.select(create_map(col('Description'), col('InvoiceNo')).alias('complex_map'))\
.selectExpr('explode(complex_map)').show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



In [101]:
# 스파크에선느 JSON으로도 다룰 수 있음 
# JSON 생성
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

jsonDF.show(1, False)

+-------------------------------------------+
|jsonString                                 |
+-------------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+



In [102]:
# get_json_object 함수로 Json 객체를 인라인 쿼리로 조회할 수 있음
# 단일 수준 JSON 객체면 json_tuple도 가능

from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),
    json_tuple(col("jsonString"), "myJSONKey")).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJSONValue":[1...|
+------+--------------------+



In [103]:
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")))

DataFrame[structstojson(myStruct): string]

In [104]:
# to_json 함수로 structtype을 json으로 변환 가능
from pyspark.sql.functions import to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")\
.select(to_json(col('myStruct')))

DataFrame[structstojson(myStruct): string]

In [105]:
# to_json 함수에 Json 데이터소스와 동일한 형태의 딕셔너리를 파라미터로 사용할 수 있음
# from_json 함수를 사용해 json 문자열을 객체로 변환할 수 있음


from pyspark.sql.functions import from_json
from pyspark.sql.types import *
parseSchema = StructType((
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("newJSON"))\
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)

+----------------------+--------------------+
|jsontostructs(newJSON)|             newJSON|
+----------------------+--------------------+
|  [536365, WHITE HA...|{"InvoiceNo":"536...|
|  [536365, WHITE ME...|{"InvoiceNo":"536...|
+----------------------+--------------------+
only showing top 2 rows



In [106]:
# 스파크는 사용자 정의 함수를 사용할 수 있다. 
# 파이썬이나 스칼라 그리고 외부 라이브러리를 사용해 사용자가 원하는 형태로 트랜스포메이션을 만들 수 있게 함. 
# 여러 프로그래밍 언어를 활용할 수 있음

# 언어별로 성능에 영향을 미칠 수 있어서 주의해야함. 

udfExampleDF=spark.range(5).toDF('num')
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

- 함수를 생성하게 되면 모든 워커 노드에 생성된 함수를 사용할 수 있도록 스파크에 등록해줘야함. 
- 드라이버에서 함수를 직렬화하고 네트워크를 통해 모든 익스큐터 프로세스를 전달함.
- 스칼라나 자바로 함수를 작성하면 JVM 환경에서만 사용할 수 있음. -> 약간의 성능 저하 발생 
- 파이썬으로 작성하면 다르게 동작함. -> 파이썬이 이해할 수 있는 포맷으로 모든 데이터를 직렬화함. 
- 파이썬 프로세스에 있는 데이터의 로우마다 함수를 실행하고 마지막으로 JVM과 스파크에 처리 결과 반환
- 부하는 파이썬 프로세스의 시작 부분에서도 일어나지만, 파이썬으로 데이터를 전달하기 위해 직렬화 하는 과정에서 많이 발생함. 
- 직렬화에 큰 부하가 발생하고, 데이터가 파이썬으로 전달되면 스파크에서 워커 메모리를 관리할 수 없음. 
- 자바나 스칼라로 함수를 작성하는 것을 추천하고 있음. 

In [107]:
# 사용자 정의 함수 등록
from pyspark.sql.functions import udf
power3udf = udf(power3)

# 등록하면 DataFrame에 사용 가능
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show(2)

# 문자열 표현식에는 사용할 수 없지만, 스칼라로 SQL 함수로 등록하면 가능해짐. 
# 스칼라에서 파이썬으로 우회하는 방식

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
+-----------+
only showing top 2 rows



In [108]:
# 파이썬으로 UDF 등록
from pyspark.sql.types import IntegerType, DoubleType
spark.udf.register("power3py", power3, DoubleType())

<function __main__.power3(double_value)>

In [109]:
# null이 나오는 이유는 rnage 메서드가 integer 데이터 타입의 데이터를 만들기 때문임. integer 대신 float이 나오게 수정해야함.
udfExampleDF.selectExpr("power3py(num)").show(2)

+-------------+
|power3py(num)|
+-------------+
|         null|
|         null|
+-------------+
only showing top 2 rows

